# Importando bibliotecas necessárias

## Para conexão com a API

Documentação requests: https://requests.readthedocs.io/en/latest/user/quickstart/

In [1]:
import requests
import os
import json
import sys
import logging
import schedule
import time
import smtplib
import win32com.client as win32
import warnings
from email.message import EmailMessage
from datetime import datetime
from pathlib import Path

## Para a automação

Utilizando o Selenium e WebDriver para acessar a Web

Documentação: https://www.selenium.dev/documentation/webdriver/

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchFrameException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains
from colorama import init, Fore

warnings.filterwarnings("ignore")
sys.tracebacklimit = 0
init(autoreset=True)

service = Service(ChromeDriverManager().install())

horario = ['06:00', '14:00', '17:00', '20:00']  

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.30M/6.30M [00:00<00:00, 10.7MB/s]


# 1 - Acessando a API do EnSpace

O primeiro passo é coletar o token para autenticação.

O token é especificado na chave 'jwt'. Válido por 24 horas.

# 2 - Acessar as tarefas do usuário (EnSpace)

Dentro do headers, deve ser passado: 'Authorization': 'token', 'enl-token': 'enspace4c4c'

Atenção: Junto ao token, o prefixo "Bearer " deve ser incluído

# 3 - Definindo as classes

# 3.0 - Email

In [3]:
class Email():
    def __init__(self, tarefas):
        self.outlook = win32.Dispatch('outlook.application')
        self.tarefas = tarefas
    
    def envia_email_sucesso(self):
        try:
            data = datetime.now()
            hoje = data.strftime("%d/%m/%Y")
            hora = data.strftime("%H:%M")
            self.email = self.outlook.CreateItem(0)
            self.email.To = "jose.bueno@be-enlighten.com; beatriz.ferreira@be-enlighten.com; priscila.condeli@be-enlighten.com; angela.moreira@be-enlighten.com"
            self.email.Subject = "O algoritmo foi iniciado!"
            self.email.Body = """
            Olá,
            
            O algoritmo foi executado às {} hrs, do dia {}, e encontrou {} tarefas.
            
            at.te,
            
            Alpha Bot
            """.format(hora, hoje, self.tarefas)
            
            self.email.Send()
        except:
            print(Fore.RED + "Não foi possível enviar email. ")

    def envia_email_fim(self, bandeira1, bandeira2, bandeira3):
        try:
            data = datetime.now()
            hora = data.strftime("%H:%M")
            self.email = self.outlook.CreateItem(0)
            self.email.To = "jose.bueno@be-enlighten.com; beatriz.ferreira@be-enlighten.com; priscila.condeli@be-enlighten.com; angela.moreira@be-enlighten.com"
            self.email.Subject = "O algoritmo finalizou a execução!"
            self.email.Body = """
            Olá,

            O algoritmo foi finalizado às {} horas e executou todas as tarefas corretamente.

            Foram executadas {} tarefas, das quais:
            •	{} foram completadas com sucesso.
            •	{} não havia registro no eLaw.
            •	{} estavam com registros incorretos ou foram encontradas várias correspondências.

            At.te,

            Alpha Bot
            """.format(hora, self.tarefas, bandeira1, bandeira2, bandeira3)

            self.email.Send()

        except:
             print(Fore.RED + "Não foi possível enviar email. ")
    
    def envia_email_erro(self):
        try:
            self.email = self.outlook.CreateItem(0)
            self.email.To = "jose.bueno@be-enlighten.com; beatriz.ferreira@be-enlighten.com"
            self.email.Subject = "Erro na execução do algoritmo!"
            self.email.HTMLBody = "<p> O algoritmo encontrou um erro e precisou ser finalizado."
            self.email.Send()
        except:
             print(Fore.RED + "Não foi possível enviar email. ")



## 3.1- API

In [4]:
class AcessaAPI():
    def __init__(self):
        self.r = ""
        self.url = ""
        self.data = ""
        self.headers = ""
        self.payload = {}
        self.token = ""
  
    def post_API(self):
        try:
            print("Tentando acessar a API...")
            self.url = 'https://api.enspace.io/auth/local'
            self.data = {"identifier": "robo.alpha@be-enlighten.com", "password": "Robô@Alpha2023"}
            self.headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
            r = requests.post(self.url, data=json.dumps(self.data), headers=self.headers)
            r = r.json()
        except Exception as e:
            print("Reposta HTTP: ", r.status_code)
            print("Reposta: ", r.reason)
            print(r.raise_for_status())
            sys.exit(Fore.RED + "\tHouve algum erro ao fazer a requisição POST da API. Encerrando o algoritmo.")
        
        print(Fore.GREEN + "\tAPI acessada com sucesso!")
        
        return r
    
    def get_API(self, r):
        print("Tentando acessar as tarefas da API.")
        try:
            self.token = "Bearer " + r['jwt']
            self.url = 'https://api.enspace.io/c-flow-item-tasks/in-charge'
            self.payload = {
                'work_status': 'waiting',
                'task.name_in': ['Analisar Documento - 48 horas', 
                                 'Analisar Documento - Acima de 15 dias',
                                 'Analisar Documento - 15 dias',
                                 'Analisar Documento - 10 dias',
                                 'Analisar Documento - 5 dias'
                                ],
                '_sort': 'id:DESC',
                '_limit': -1,
                '__relations': 'flow_item.item'
            }

            self.headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'Authorization':self.token, 'enl-token': 'enspace4c4c'}
            r = requests.get(self.url, headers = self.headers, params = self.payload)
            r = r.json()
        except Exception as e:
            print("Reposta HTTP: ", r.status_code)
            print("Reposta: ", r.reason)
            print(r.raise_for_status())
            sys.exit(Fore.RED + "\tHouve algum erro ao fazer a requisição GET da API. Encerrando o algoritmo.")
        
        print(Fore.GREEN + "\tToken coletado e dados do JSON retornados.")
        
        return r
    
    def altera_status(self, id_tarefa):
        print("\nAlterando o status da tarefa...")
        try:
            self.url = 'https://api.enspace.io/c-flow-item-tasks/{}/assign'.format(id_tarefa)
            self.headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'Authorization':self.token, 'enl-token': 'enspace4c4c'}
            
            retorno = requests.post(self.url, headers = self.headers)
            
            if retorno.status_code != 200:
                print("Resposta HTTP: ", retorno.status_code)
                print("Resposta: ", retorno.reason)
                print(retorno.raise_for_status())
                print(Fore.RED + "\n A API rejeitou o envio dos dados. Favor verificar.")
                return
            else:
                print("Resposta HTTP: ", retorno.status_code)
                print("\tStatus da tarefa alterado: Waiting -> Working.")
            
        except requests.exceptions.HTTPError as err:
            print("HTTP Error")
            print(err.args[0])

    def tarefa_completa(self, id_tarefa, id_elaw):
        print("\nRetornando dados para a API...")
        try:
            #Converte a data de agora para padrão JSON
            current_date = datetime.now()
            output_date = current_date.strftime("%Y-%m-%dT%H:%M")
            self.url = 'https://api.enspace.io/c-flow-item-tasks/{}/complete'.format(id_tarefa)
            self.headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'Authorization':self.token, 'enl-token': 'enspace4c4c'}
            self.data = {
                'tipo_de_demanda': 'Cadastro Elaw',
                'tipo_de_cadastro': 'Up to Date',
                'data_conclusao_cadastro_elaw': output_date,
                'id_elaw': id_elaw,
                'demanda_ja_tratada_em_outro_ticket': 'Não',
                'cadastro_pelo_robo': 'Sim'
                }

            retorno = requests.put(self.url, data=json.dumps(self.data), headers = self.headers)

            if retorno.status_code != 200:
                print(retorno)
                print("Resposta HTTP: ", retorno.status_code)
                print("Resposta: ", retorno.reason)
                print(retorno.raise_for_status())
                print(Fore.RED + "\n A API rejeitou o envio dos dados. Favor verificar.")
                return
            else:
                print("Resposta HTTP: ", retorno.status_code)
                print(Fore.GREEN + "\tOs dados foram salvos com sucesso!")
                print("------------------------------------------- ")
            
        except requests.exceptions.HTTPError as err:
            print("HTTP Error")
            print(err.args[0])
        
    
    def tarefa_incompleta(self, referencia):
        try:
            self.url = 'https://api.enspace.io/c-items/{}'.format(referencia)
            self.headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'Authorization':self.token, 'enl-token': 'enspace4c4c'}
            self.data = {'data':{'cadastro_pelo_robo': 'Não', 'observacoes':'Não foi encontrado nenhum registro para a tarefa.'}}
                        
                        
            retorno = requests.put(self.url, data=json.dumps(self.data), headers = self.headers)
            if retorno.status_code != 200:
                print(retorno)
                print("Resposta HTTP: ", retorno.status_code)
                print("Resposta: ", retorno.reason)
                print(retorno.raise_for_status())
                print(Fore.RED + "\n A API rejeitou o envio dos dados. Favor verificar.")
                return
            else:
                print("Resposta HTTP: ", retorno.status_code)

        except Exception as e:
            sys.exit(Fore.RED + "\tHouve algum erro na requisição PUT da API. Favor verificar!")
        
    def tarefa_erro(self, referencia, desc_erro):
        try:
            self.url = 'https://api.enspace.io/c-items/{}'.format(referencia)
            self.headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'Authorization':self.token, 'enl-token': 'enspace4c4c'}
            self.data = {'data':{'cadastro_pelo_robo': 'Não', 'observacoes': desc_erro}}
                      
            retorno = requests.put(self.url, data=json.dumps(self.data), headers = self.headers)
            
            if retorno.status_code != 200:
                print(retorno)
                print("Resposta HTTP: ", retorno.status_code)
                print("Resposta: ", retorno.reason)
                print(retorno.raise_for_status())
                print(Fore.RED + "\n A API rejeitou o envio dos dados. Favor verificar.")
                return
            else:
                print("Resposta HTTP: ", retorno.status_code)

        except Exception as e:
            sys.exit(Fore.RED + "\tHouve algum erro na requisição PUT da API. Favor verificar!")
            
    def tarefa_duplicada(self, id_tarefa, id_elaw):
        print("\nTarefa duplicada...retornando dados para a API.")
        try:
            #Converte a data de agora para padrão JSON
            current_date = datetime.now()
            output_date = current_date.strftime("%Y-%m-%dT%H:%M")
            self.url = 'https://api.enspace.io/c-flow-item-tasks/{}/complete'.format(id_tarefa)
            self.headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'Authorization':self.token, 'enl-token': 'enspace4c4c'}
            self.data = {
                'tipo_de_demanda': 'Cadastro Elaw',
                'tipo_de_cadastro': 'Up to Date',
                'data_conclusao_cadastro_elaw': output_date,
                'id_elaw': id_elaw,
                'demanda_ja_tratada_em_outro_ticket': 'Não',
                'cadastro_pelo_robo': 'Sim',
                'observacoes':'Já havia uma audiência cadastrada com o mesmo tipo e a mesma data.'
                }

            retorno = requests.put(self.url, data=json.dumps(self.data), headers = self.headers)
            print("Resposta HTTP: ", retorno.status_code)
            print(Fore.GREEN + "\tOs dados foram salvos com sucesso!")
            print("------------------------------------------- ")
        except requests.exceptions.HTTPError as err:
            print(retorno)
            print("HTTP Error")
            print(err.args[0])
        

## 3.2 - Classe do Selenium

In [5]:
class Automacao():
    def __init__(self):
        self.navegador = ""
    
    def InicializaWebDriver(self):
        try:
            chrome_options = Options()
            chrome_options.add_experimental_option("detach", True)
            self.navegador = webdriver.Chrome(service=service, chrome_options=chrome_options)
            self.navegador.implicitly_wait(30)
            self.navegador.get("https://mercadolivre.elaw.com.br/")
            print("Tentando acessar o navegador...")
            #time.sleep(5)
        except Exception as e:
            sys.exit(Fore.RED + "Não foi possível incializar o método Webdriver. Encerrando o algoritmo.")
        
        print(Fore.GREEN + "\tNavegador acessado com sucesso!")

    def AcessaElaw(self):
        print("Acessando o eLaw...")
        acesso = True
        while acesso:
            try:
                self.navegador.find_element('id', 'username' ).send_keys('ext_roalpha')
                #time.sleep(1)
                self.navegador.find_element('id', 'password').send_keys('Alpha@2023')
                #time.sleep(1)
                self.navegador.find_element('id', 'j_id_a_1_6_h_2_8').click()
                #time.sleep(10)
                self.navegador.find_element('xpath', '//*[@id="j_id_2d_1"]/ul/li[2]/a').click()
                acesso = False
            except Exception as e:
                print(Fore.RED + "Não foi possível acessar o eLaw. Tentando novamente...")
                print("------------------------------------------- ")
        
        print(Fore.GREEN + "\tElaw acessado com sucesso!")
        
        #time.sleep(1)
        self.navegador.find_element('xpath', '//*[@id="menu-form-contencioso:j_id_2d_a_4"]/a').click()
        
    def RetornaObjeto(self):
        return self.navegador

## 3.3 - Notificação

In [6]:
class TipoNotificao():
    def __init__(self, i, r, navegador):
        self.i = i
        self.r = r
        self.navegador = navegador
        self.nomes = []
        self.urls = []
        self.bandeira = ""
        self.id_elaw = ""
        self.item = ""
        self.desc_erro = ""
    
    def return_bandeira(self):
        return self.bandeira
    
    def return_id_elaw(self):
        return self.id_elaw
    
    def return_id_tarefa(self):
        return self.id_tarefa
    
    def return_item_tarefa(self):
        return self.item
    
    def return_desc_erro(self):
        return self.desc_erro
    
    #Serve para ambos
    def voltaPaginaInicial(self):
        self.navegador.refresh()
        time.sleep(2)
        wait(self.navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="j_id_2d_1"]/ul/li[2]/a'))).click()       
        wait(self.navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="menu-form-contencioso:j_id_2d_a_4"]/a'))).click()
        wait(self.navegador, 10).until(EC.element_to_be_clickable((By.ID, 'tabSearchTab:txtSearch'))).clear()
    
    #Serve para ambos    
    def trocaAdvogado(self):
        try:
            escritorio_advogado = {
                    'GONDIM ADVOGADOS':'Liana Lopes Martins',
                    'CHALFIN, GOLDBERG E VAINBOIM':'Cristina Tsiftzoglou',
                    'DANNEMANN SIEMSEN ADVOGADOS':'Bruno Wermelinger de Oliveira',
                    'MARTORELLI ADVOGADOS':'Kamila Costa de Miranda',
                    'ASPIS & PALMEIRO DA FONTOURA ADVOGADOS ASSOCIADOS':'Luiza Cardias',
                    'VOSGERAU & CUNHA ADVOGADOS ASSOCIADOS':'Bruno Roberto Vosgerau',
                    'LIMA FEIGELSON ADVOGADOS':'Laís Arruda Marini',
                    'ANDRADE MAIA ADVOGADOS':'Alessandra Nazareth Mottini',
                    'OLIVEIRA RAMOS ADVOGADOS':'Alyne Aparecida Guimarães dos Santos',
                    'BBL Advogados':'João Pedro Brígido Pinheiro da Silva',
                    'FINCH SOLUÇÕES':'DAIANE VIAN DOS SANTOS',
                    'Morais Andrade':'Marcella Porcelli',
                    'Chalfin - Trabalhista':'Pamella Maria Fernandes Iglesias Silva Abreu',
                    'Chalfin - Diligências': 'Layane Dantas Formiga',
                    'ERNESTO BORGES ADVOGADOS':'Thaísa Ferreira',
                    'GAIA, SILVA, GAEDE & ASSOCIADOS': 'Maria Aline Buratto Aun',
                    'Goulart Penteado': 'Victoria Campanha',
                    'MACHADO MEYER SENDACZ OPICE': 'Daniela Leme Arca',
                    'MURTA GOYANES ADVOGADOS': 'Gabriel Monnerat Cyrino da Gama e Silva',
                    'OPICE BLUM BRUNO ABRUSIO VAINZOF': 'Fernanda Martins Miranda',
                    'Ouvidoria PROCON': 'Carolina Aguiar Franco Da Veiga',
                    'Pimentel Advogados': 'Daniel Cunha Canto Marques',
                    'Trench Rossi Watanabe': 'Marcelo Alves de Siqueira',
                    'Rangel e Simões':'Mariana Del Monaco'
                }

            nome_escritorio = self.navegador.find_elements('xpath', '//*[@id="processoDadosCabecalhoForm"]/table/tbody/tr/td/label')
            for escritorio in escritorio_advogado:
                for nome in nome_escritorio:
                    if escritorio == nome.text:
                        advogado = escritorio_advogado[escritorio]
                        break
        except:
            print(Fore.RED + "Não foi encontrado advogado.")
            self.bandeira = "Erro2"
            self.desc_erro = "Não foi encontrado advogado."
            self.voltaPaginaInicial()
            return
            
            print('\tAdvogado responsável:', advogado)
        
        time.sleep(1)
        self.navegador.find_element(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_4_1_3_d:dtAgendamentoResults:0:j_id_i3_4_1_3_1g"]').click()
        
        time.sleep(5)
        try:
            self.navegador.switch_to.frame(1)
        except NoSuchFrameException:
            print(Fore.RED + "Frame advogado demorou muito para carregar.")
            self.bandeira = "Erro2"
            self.desc_erro = "Frame advogado demorou muito para carregar."
            self.voltaPaginaInicial()
            return
    
        total_advogados = self.navegador.find_elements(By.XPATH, '//*[@id="dtLawyerParticipantesProcessoResults_data"]/tr')

        time.sleep(2)

        #botao excluir
        for i in range(len(total_advogados)):
            a = 0
            #self.navegador.find_element(By.ID, 'dtLawyerParticipantesProcessoResults:{0}:j_id_1b').click()
            self.navegador.find_element(By.ID,  'dtLawyerParticipantesProcessoResults:0:j_id_1b').click()
            time.sleep(3)
            a += 1
            if a == 3:
                a = 0
                time.sleep(1)
        
        time.sleep(1)
        self.navegador.find_element(By.ID, 'dtLawyerParticipantesProcessoResults:autoCompleteLawyer_input').send_keys(advogado)

        time.sleep(1)
        self.navegador.find_element(By.XPATH, '//*[@id="dtLawyerParticipantesProcessoResults:autoCompleteLawyer_panel"]/ul').click()

        time.sleep(1)
        self.navegador.find_element(By.ID, 'comboAdvogadoResponsavelProcesso_label').click()

        time.sleep(1)
        self.navegador.find_element(By.ID, 'comboAdvogadoResponsavelProcesso_1').click()

        #Confirma
        time.sleep(1)
        self.navegador.find_element(By.ID, 'j_id_t').click()
    
    #Serve para ambos
    def ColetaDados(self):
        self.item = self.r[self.i]['flow_item']['item']['reference']
        self.tipo_providencia = self.r[self.i]['flow_item']['item']['data']['tipo_de_providencia']
        self.id_tarefa = self.r[self.i]['id']
        self.status_tarefa = self.r[self.i]['work_status']
        self.task = self.r[self.i]['task_name']
        self.tipo_processo = self.r[self.i]['flow_item']['item']['data']['tipo_de_processo']
        self.prazo = self.r[self.i]['flow_item']['item']['data']['prazo_2957']
        
        try:
            self.num_reclamacao_procon = self.r[self.i]['flow_item']['item']['data']['numero_da_fa_cip']
            self.numero = self.num_reclamacao_procon
        except:
            self.num_processo = self.r[self.i]['flow_item']['item']['data']['numero_do_processo']
            self.numero = self.num_processo
        
        try:
            for anexo in self.r[self.i]['flow_item']['item']['data']['anexar_notificacao']:
                self.nomes.append(anexo['name'])
                if not 'https' in anexo['url']:
                    anexo['url'] = 'https://' + anexo['url']
                
                self.urls.append(anexo['url'])
        except:
            pass
        
        print("\nDados de NOTIFICAÇÃO coletados com sucesso!")
        
    #Serve para ambos    
    def MostraDados(self):
        print("\nReferência do Item: ", self.item)
        print("Tipo de providência: ", self.tipo_providencia)
        print("ID da Tarefa: ", self.id_tarefa)
        print('Status da Tarefa: ', self.status_tarefa)
        print('Task: ', self.task)
        print("Tipo de processo: ", self.tipo_processo)
        print("Número da reclamação procon/numero processo: ", self.numero)
        print("Prazo: ", self.prazo)
        print("Nome do arquivo: ", self.nomes)
        print("URls: ", self.urls)
        
    def PreencheDados(self):
        encontrou = True
        while encontrou:
            try:
                #Insere o processo
                self.navegador.find_element(By.ID, 'tabSearchTab:txtSearch').send_keys(self.numero)

                #Clica em pesquisar
                self.navegador.find_element(By.ID, 'btnPesquisar').click()
                encontrou = False
            except NoSuchElementException:
                print("Tentando encontrar o botão novamente...")
                self.voltaPaginaInicial()
        
        #Verifica se existe o processo
        total = self.navegador.find_elements(By.CSS_SELECTOR, 'tr[role = row]')
        try:
            if len(total) <= 1:
                print("\nNão foi encontrado nenhum registro para o ID: ", self.id_tarefa)
                print("------------------------------------------- ")
                self.navegador.find_element('id', 'tabSearchTab:txtSearch').clear()
                self.bandeira = "Erro1"
                return
            elif len(total) > 2:
                print("\nO algoritmo encontrou várias correspondências para o ID: ", self.id_tarefa, ".Indo para o próximo.")
                print("------------------------------------------- ")
                self.navegador.find_element('id', 'tabSearchTab:txtSearch').clear()
                self.desc_erro = "O algoritmo encontrou várias correspondências para essa tarefa."
                self.bandeira = "Erro2"
                return
            else:
                time.sleep(1)
                #Encontrar o id do Elaw
                self.id_elaw = self.navegador.find_element(By.XPATH,'//*[@id="dtProcessoResults:0:j_id_1hs:0:j_id_1hw"]/span').text
                
                #Verifica se os números são iguais
                num_processo = navegador.find_element(By.XPATH, '//*[@id="dtProcessoResults:0:j_id_1hs:5:j_id_1hw"]/span').text
                if not num_processo == self.numero:
                    print("Os números de processo não são iguais. Indo para o próximo")
                    print("------------------------------------------- ")
                    self.navegador.find_element('id', 'tabSearchTab:txtSearch').clear()
                    self.bandeira = "Erro2"
                    self.desc_erro = "O número do processo diverge do cadastrado no eLaw."
                    return
                
                #Clicar em pesquisar
                self.navegador.find_element(By.ID, 'dtProcessoResults:0:btnProcesso').click()
        except:
            print(Fore.RED + "Não foi possível encontrar métricas do algoritmo. Encerrando.")
            self.bandeira = "Erro2"
            self.desc_erro = "O servidos do Elaw não respondeu."
            self.voltaPaginaInicial()
            return
            
        
        #Verifica status
        status_label = self.navegador.find_elements('xpath', '//*[@id="processoDadosCabecalhoForm"]/table/tbody/tr/td/label')
        for status in status_label:
            if status.text == "Encerrado":
                self.navegador.find_element(By.ID, 'btnTrocarStatus').click()
                wait(self.navegador, 10).until(EC.visibility_of_element_located((By.ID, 'trocarStatusDialog')))
                time.sleep(1)
                self.navegador.find_element(By.ID, 'comboStatus_label').click()
                time.sleep(1)
                self.navegador.find_element(By.ID, 'comboStatus_2').click()
                time.sleep(1)
                self.navegador.find_element(By.ID, 'j_id_fk').click()
        
        #Verifica advogado
        time.sleep(3)
        linhas = self.navegador.find_elements(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_4_1_3_d:dtAgendamentoResults_data"]/tr')
        
        time.sleep(3)
        print(linhas[0].text)
        if not linhas[0].text == "Nenhum registro encontrado!":
            self.trocaAdvogado()
        
        #Até aqui serve para ambos
        
        #Clica em Acionar Workflow
        time.sleep(2)
        t = 0
        botao = True
        while botao:
            if not t == 3:
                try:
                    time.sleep(2)
                    wait(self.navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnAcionarWorkflow"]/span[2]'))).click()

                    #Alterando para o popup
                    wait(self.navegador, 10).until(EC.visibility_of_element_located((By.ID, "acionarWorkflowDialog")))
                    botao = False
                except:
                    t += 1
                    print(Fore.RED + "O botão do modal não foi encontrado. Tentando novamente...")
                    self.navegador.refresh()
                    time.sleep(5)
            else:
                print(Fore.RED + "O botão do modal não respondeu. Indo para o próximo")
                self.bandeira = "Erro2"
                self.desc_erro = "Frame advogado demorou muito para carregar."
                self.voltaPaginaInicial()
                botao = False
                return
                
        #Inserindo informações no popup
        t = 0
        botao = True
        while botao:
            if not t == 3:
                try:
                    time.sleep(1)
                    self.navegador.find_element(By.ID, 'j_id_2n_label').click()
                    #wait(self.navegador, 10).until(EC.element_located_to_be_selected((By.ID, 'j_id_2n_label'))).click()

                    time.sleep(1)
                    self.navegador.find_element(By.ID, 'j_id_2n_12').click()

                    time.sleep(3)
                    self.navegador.find_element(By.ID, 'workflowFaseAcionarWorkflowCombo_label').click()
                    #wait(self.navegador, 10).until(EC.element_located_to_be_selected((By.ID, 'workflowFaseAcionarWorkflowCombo_label'))).click()

                    time.sleep(1)
                    self.navegador.find_element(By.ID, 'workflowFaseAcionarWorkflowCombo_1').click()
                    #wait(self.navegador, 10).until(EC.element_located_to_be_selected((By.ID, 'workflowFaseAcionarWorkflowCombo_1'))).click()
                    botao = False
                except NoSuchElementException:
                    t += 1
                    print("Botão não encontrado... Tentando novamente")
            else:
                print(Fore.RED + "O botão do modal não respondeu. Indo para o próximo")
                self.bandeira = "Erro2"
                self.desc_erro = "Botão de acionar workflow demorou muito para responder."
                self.voltaPaginaInicial()
                botao = False
                return
        
        #Espera mais
        try:
            self.navegador.implicitly_wait(15)
            #Clica em confirmar
            time.sleep(1)
            wait(self.navegador, 10).until(EC.element_to_be_clickable((By.ID, 'j_id_3e'))).click()
        
            #Clicar em Tipo - Notificação no anexo dos arquivos
            self.navegador.find_element(By.ID, 'j_id_78_2_1_5_5b_1:eFileTipoCombo_label').click()
            time.sleep(1)
            self.navegador.find_element(By.ID, 'j_id_78_2_1_5_5b_1:eFileTipoCombo_32').click()
        except: 
            #Volta para a tela inicial
            print(Fore.RED + "Não foi carregado o botão. Indo para o próximo")
            self.bandeira = "Erro2"
            self.desc_erro = "Botão de anexar arquivo demorou muito para responder."
            self.voltaPaginaInicial()
            return
            
        time.sleep(1)
        #Baixando os arquivos
        try:
            for pos, i in enumerate(self.urls):
                print("Baixando o arquivo: ", self.nomes[pos])
                #filename = Path(r"C:\Users\automation\Downloads\{}".format(self.nomes[pos]))
                filename = Path(r"C:\Users\JoséGabrielNevesBuen\Downloads\{}".format(self.nomes[pos]))
                arquivo_pdf = requests.get(self.urls[pos])
                filename.write_bytes(arquivo_pdf.content)
                time.sleep(1)
                #self.navegador.find_element('id', 'j_id_78_2_1_5_5b_1:j_id_78_2_1_5_5b_3_2_e_2_1_input').send_keys(r"C:\Users\automation\Downloads\{}".format(self.nomes[pos]))
                self.navegador.find_element(By.ID, 'j_id_78_2_1_5_5b_1:j_id_78_2_1_5_5b_3_2_e_2_1_input').send_keys(r"C:\Users\JoséGabrielNevesBuen\Downloads\{}".format(self.nomes[pos]))
        except:
            print(Fore.RED + "O servidor demorou para responder a requisição de download. Indo para a próxima tarefa.")
            self.bandeira = "Erro2"
            self.desc_erro = "O servidor demorou para responder a requisição de download."
            self.voltaPaginaInicial()
            return

        #Clica em enviar
        time.sleep(2)
        try:
            self.navegador.find_element(By.ID, 'btnConfirmaSim').click()
        except:
            print("Não foi carregado o botão. Indo para o próximo")
            self.bandeira = "Erro2"
            self.desc_erro = "O botão de confirmar demorou muito para responder."
            self.voltaPaginaInicial()
            return
        
        try:
            #Volta para a tela inicial
            botao = True
            while botao:
                if not t == 5:
                    try:
                        time.sleep(1)
                        self.navegador.find_element('xpath', '//*[@id="j_id_2d_1"]/ul/li[2]/a').click()
                        botao = False
                    except:
                        t += 1
                else:
                    time.sleep(3)
                    print(Fore.RED + "O botão de iniciar demorou muito para carregar")
                    self.bandeira = "Erro2"
                    self.desc_erro = "O botão de iniciar demorou muito para carregar."
                    self.voltaPaginaInicial()
                    botao = False
                    return
            
            time.sleep(2)
            wait(self.navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="menu-form-contencioso:j_id_2d_a_4"]/a'))).click()
            time.sleep(2)
            wait(self.navegador, 10).until(EC.element_to_be_clickable((By.ID, 'tabSearchTab:txtSearch'))).clear()
        except TimeoutException as TME:
            print(Fore.RED + "O servidor demorou muito para responder.")
            self.bandeira = "Erro2"
            self.desc_erro = "O servidor demorou muito para responder"
            self.voltaPaginaInicial()
            return



## 3.4 - Audiência

In [7]:
class TipoAudiencia():
    def __init__(self, i, r, navegador):
        self.i = i
        self.r = r
        self.navegador = navegador
        self.nomes = []
        self.urls = []
        self.bandeira = ""
        self.id_elaw = ""
        self.item = ""
        self.desc_erro = ""
    
    def return_bandeira(self):
        return self.bandeira
    
    def return_id_elaw(self):
        return self.id_elaw
    
    def return_id_tarefa(self):
        return self.id_tarefa
    
    def return_item_tarefa(self):
        return self.item
    
    def return_desc_erro(self):
        return self.desc_erro
    
    def voltaPaginaInicial(self):
        self.navegador.refresh()
        wait(self.navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="j_id_2d_1"]/ul/li[2]/a'))).click()       
        wait(self.navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="menu-form-contencioso:j_id_2d_a_4"]/a'))).click()
        wait(self.navegador, 10).until(EC.element_to_be_clickable((By.ID, 'tabSearchTab:txtSearch'))).clear()
        
    def trocaAdvogado(self):
        try: 
            #Encontra o escritório    
            escritorio_advogado = {
                'GONDIM ADVOGADOS':'Liana Lopes Martins',
                'CHALFIN, GOLDBERG E VAINBOIM':'Cristina Tsiftzoglou',
                'DANNEMANN SIEMSEN ADVOGADOS':'Bruno Wermelinger de Oliveira',
                'MARTORELLI ADVOGADOS':'Kamila Costa de Miranda',
                'ASPIS & PALMEIRO DA FONTOURA ADVOGADOS ASSOCIADOS':'Luiza Cardias',
                'VOSGERAU & CUNHA ADVOGADOS ASSOCIADOS':'Bruno Roberto Vosgerau',
                'LIMA FEIGELSON ADVOGADOS':'Laís Arruda Marini',
                'ANDRADE MAIA ADVOGADOS':'Alessandra Nazareth Mottini',
                'OLIVEIRA RAMOS ADVOGADOS':'Alyne Aparecida Guimarães dos Santos',
                'BBL Advogados':'João Pedro Brígido Pinheiro da Silva',
                'FINCH SOLUÇÕES':'DAIANE VIAN DOS SANTOS',
                'Morais Andrade':'Marcella Porcelli',
                'Chalfin - Trabalhista':'Pamella Maria Fernandes Iglesias Silva Abreu',
                'Chalfin - Diligências': 'Layane Dantas Formiga',
                'ERNESTO BORGES ADVOGADOS':'Thaísa Ferreira',
                'GAIA, SILVA, GAEDE & ASSOCIADOS': 'Maria Aline Buratto Aun',
                'Goulart Penteado': 'Victoria Campanha',
                'MACHADO MEYER SENDACZ OPICE': 'Daniela Leme Arca',
                'MURTA GOYANES ADVOGADOS': 'Gabriel Monnerat Cyrino da Gama e Silva',
                'OPICE BLUM BRUNO ABRUSIO VAINZOF': 'Fernanda Martins Miranda',
                'Ouvidoria PROCON': 'Carolina Aguiar Franco Da Veiga',
                'Pimentel Advogados': 'Daniel Cunha Canto Marques',
                'Trench Rossi Watanabe': 'Marcelo Alves de Siqueira',
                'Rangel e Simões':'Mariana Del Monaco'
            }
            
            time.sleep(2)
            nome_escritorio = self.navegador.find_elements('xpath', '//*[@id="processoDadosCabecalhoForm"]/table/tbody/tr/td/label')
            for escritorio in escritorio_advogado:
                for nome in nome_escritorio:
                    if escritorio == nome.text:
                        advogado = escritorio_advogado[escritorio]
                        break
          
        except:
            print(Fore.RED + "Não foi encontrado o advogado para o escritório: ", nome.text)
            self.bandeira = "Erro2"
            self.desc_erro = "Campo do escritório do advogado vazio."
            self.voltaPaginaInicial()
            return
        
        try:
            time.sleep(1)
            self.navegador.find_element(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_4_1_3_d:dtAgendamentoResults:0:j_id_i3_4_1_3_1g"]').click()

            time.sleep(2)
            self.navegador.switch_to.frame(1)

            total_advogados = self.navegador.find_elements(By.XPATH, '//*[@id="dtLawyerParticipantesProcessoResults_data"]/tr')

            time.sleep(2)

            #botao excluir
            for i in range(len(total_advogados)):
                a = 0
                self.navegador.find_element(By.ID, f'dtLawyerParticipantesProcessoResults:{a}:j_id_1b').click()
                a += 1
                if a == 3:
                    a = 0
                time.sleep(1)

            self.navegador.find_element(By.ID, 'dtLawyerParticipantesProcessoResults:autoCompleteLawyer_input').send_keys(advogado)

            time.sleep(1)
            self.navegador.find_element(By.XPATH, '//*[@id="dtLawyerParticipantesProcessoResults:autoCompleteLawyer_panel"]/ul').click()

            time.sleep(1)
            self.navegador.find_element(By.ID, 'comboAdvogadoResponsavelProcesso_label').click()

            time.sleep(1)
            self.navegador.find_element(By.ID, 'comboAdvogadoResponsavelProcesso_1').click()

            #Confirma
            time.sleep(1)
            self.navegador.find_element(By.ID, 'j_id_t').click()
        
        except:
            print(Fore.RED + "Botão de trocar advogado não encontrado. Indo para o próximo.")
            self.bandeira = "Erro2"
            self.desc_erro = "Botão de trocar advogado não carregado."
            self.voltaPaginaInicial()
            return
           
        
    def ColetaDados(self):
        self.item = self.r[self.i]['flow_item']['item']['reference']
        self.tipo_providencia = self.r[self.i]['flow_item']['item']['data']['tipo_de_providencia']
        self.id_tarefa = self.r[self.i]['id']
        self.status_tarefa = self.r[self.i]['work_status']
        self.task = self.r[self.i]['task_name']
        self.tipo_processo = self.r[self.i]['flow_item']['item']['data']['tipo_de_processo']
        self.prazo = self.r[self.i]['flow_item']['item']['data']['prazo_2957']
        self.tipo_audiencia = self.r[self.i]['flow_item']['item']['data']['tipo_de_audiencia']
        self.data_audiencia = self.r[self.i]['flow_item']['item']['data']['data_da_audiencia']
        
        try:
            self.num_reclamacao_procon = self.r[self.i]['flow_item']['item']['data']['numero_da_fa_cip']
            self.numero = self.num_reclamacao_procon
        except:
            self.num_processo = self.r[self.i]['flow_item']['item']['data']['numero_do_processo']
            self.numero = self.num_processo
        
        try:
            for anexo in self.r[self.i]['flow_item']['item']['data']['anexar_notificacao']:
                self.nomes.append(anexo['name'])
                if not 'https' in anexo['url']:
                    anexo['url'] = 'https://' + anexo['url']
                
                self.urls.append(anexo['url'])
        except:
            print("Nenhum arquivo foi encontrado.")
        
        print("\nDados de AUDIÊNCIA coletados com sucesso!")
        
     
    def MostraDados(self):
        print("\nReferência do Item: ", self.item)
        print("Tipo de providência: ", self.tipo_providencia)
        print("ID da Tarefa: ", self.id_tarefa)
        print('Status da Tarefa: ', self.status_tarefa)
        print('Task: ', self.task)
        print("Tipo de processo: ", self.tipo_processo)
        print("Número da reclamação procon/numero processo: ", self.numero)
        print("Tipo de Audiência: ", self.tipo_audiencia)
        print("Data da Audiência: ", self.data_audiencia)
        print("Prazo: ", self.prazo)
        print("Nome do arquivo: ", self.nomes)
        print("URls: ", self.urls)

    def PreencheDados(self):
        encontrou = True
        while encontrou:
            try:
                #Insere o processo
                self.navegador.find_element(By.ID, 'tabSearchTab:txtSearch').send_keys(self.numero)

                #Clica em pesquisar
                self.navegador.find_element(By.ID, 'btnPesquisar').click()
                encontrou = False
                
            except NoSuchElementException:
                print("Tentando encontrar o botão novamente...")
                self.voltaPaginaInicial()
        
        #Verifica se existe casos
        total = self.navegador.find_elements(By.CSS_SELECTOR, 'tr[role = row]')
        try:
            if len(total) <= 1:
                print("\nNão foi encontrado nenhum registro para o ID: ", self.id_tarefa)
                print("------------------------------------------- ")
                self.navegador.find_element(By.ID, 'tabSearchTab:txtSearch').clear()
                self.bandeira = "Erro1"
                return
            elif len(total) > 2:
                print("\nO algoritmo encontrou várias correspondências para o ID: ", self.id_tarefa, ".Indo para o próximo.")
                print("------------------------------------------- ")
                self.navegador.find_element(By.ID, 'tabSearchTab:txtSearch').clear()
                self.desc_erro = "O algoritmo encontrou várias correspondências para esse ID."
                self.bandeira = "Erro2"
                return
            else:
                #Verifica se os números são iguais
                num_processo = navegador.find_element(By.XPATH, '//*[@id="dtProcessoResults:0:j_id_1hs:5:j_id_1hw"]/span').text
                if not num_processo == self.numero:
                    print("Os números de processo não são iguais. Indo para o próximo")
                    print('-------------------------------------------')
                    self.navegador.find_element('id', 'tabSearchTab:txtSearch').clear()
                    self.bandeira = "Erro2"
                    return
                
                #Encontrar o id do Elaw
                self.id_elaw = self.navegador.find_element(By.XPATH, '//*[@id="dtProcessoResults:0:j_id_1hs:0:j_id_1hw"]/span').text
        
                #Clicar em pesquisar
                time.sleep(2)
                self.navegador.find_element(By.ID, 'dtProcessoResults:0:btnProcesso').click()
        except:
            print(Fore.RED + "Não foi possível encontrar métricas do algoritmo.")
            self.bandeira = "Erro2"
            self.desc_erro = "O servidos do Elaw não respondeu."
            self.voltaPaginaInicial()
            return
        
        #Trata a hora
        try:
            input_data = self.r[self.i]['flow_item']['item']['data']['data_da_audiencia']
            input_format = "%Y-%m-%dT%H:%M"
            output_format = "%d/%m/%Y %H:%M"
            dt = datetime.strptime(input_data, input_format)
            hora_br = dt.strftime(output_format)
        except:
            print(Fore.RED + "Não foi possível converter a hora. Digitada erroneamente: {}".format(input_data))
            self.bandeira = "Erro2"
            self.desc_erro = "Não foi possível converter a hora: {}.".format(input_data)
            self.voltaPaginaInicial()
            return
        
        #Verifica duplicidade
        time.sleep(2)
        total = self.navegador.find_elements(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_4_1_3_d:dtAgendamentoResults_data"]/tr')
        print(len(total))

        if len(total) != 1:
            for i in range(1, len(total)+1):
                try:
                    tipo = self.navegador.find_element(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_4_1_3_d:dtAgendamentoResults_data"]/tr[{}]/td[6]'.format(i)).text
                    subtipo = self.navegador.find_element(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_4_1_3_d:dtAgendamentoResults_data"]/tr[{}]/td[7]'.format(i)).text
                    data = self.navegador.find_element(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_4_1_3_d:dtAgendamentoResults_data"]/tr[{}]/td[5]'.format(i)).text
                    
                    if tipo == self.tipo_audiencia:
                        if data == self.data_audiencia:
                            if subtipo == self.tipo_processo:
                                print("O algoritmo encontrou tarefas repetidas. Indo para a próxima tarefa.")
                                self.bandeira = "Erro3"
                                return
                except:
                    print("Não há registros.")
        else:
            print("Tarefa sem duplicidade.")
        
        #Verifica status
        status_label = self.navegador.find_elements('xpath', '//*[@id="processoDadosCabecalhoForm"]/table/tbody/tr/td/label')
        for status in status_label:
            if "Encerrado" == status.text:
                self.navegador.find_element(By.ID, 'btnTrocarStatus').click()
                wait(self.navegador, 10).until(EC.visibility_of_element_located((By.ID, 'trocarStatusDialog')))
                time.sleep(2)
                self.navegador.find_element(By.ID, 'comboStatus_label').click()
                time.sleep(2)
                self.navegador.find_element(By.ID, 'comboStatus_2').click()
                time.sleep(1)
                #self.navegador.find_element('id', 'j_id_fk').click()
                self.navegador.find_element(By.XPATH, '//*[@id="trocarStatusDialog"]/div[1]/a').click()
        
        #Verifica advogado
        time.sleep(2)
        linhas = navegador.find_elements(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_4_1_3_d:dtAgendamentoResults_data"]/tr')
        
        print(linhas[0].text)
        if not linhas[0].text == "Nenhum registro encontrado!":
            self.trocaAdvogado()
       
        #Clica em "Nova Audiência"
        time.sleep(3)
        self.navegador.find_element(By.ID, 'tabViewProcesso:j_id_i3_4_1_3_8').click()
                                           
        #O loop deve acontecer aqui. Vai procurar o id correto do select
        id_select = ['j_id_2l:comboTipoAudiencia_1', 'j_id_2l:comboTipoAudiencia_2', 'j_id_2l:comboTipoAudiencia_3', 'j_id_2l:comboTipoAudiencia_4', 'j_id_2l:comboTipoAudiencia_5']
        id_select_name = ['Conciliação', 'Inicial', 'Instrução', 'Oitiva de Testemunha', 'Una']
        for index, select in enumerate(id_select):
            if self.r[self.i]['flow_item']['item']['data']['tipo_de_audiencia'] == id_select_name[index]:
                id_path = select
                break
        
        try:
            #Clica no combobox
            self.navegador.find_element(By.ID, 'j_id_2l:comboTipoAudiencia').click()

            #Clica no tipo de audiência correto
            self.navegador.find_element(By.ID, id_path).click()

            #Insere a hora
            self.navegador.find_element(By.ID, 'j_id_2l:j_id_2p_2_8_8:dataAudienciaField_input').send_keys(hora_br)
            
            '''
            if id_select_name[index] == "Oitiva de Testemunha":
                #Insere o advogado
                mensagem = self.navegador.find_element(By.XPATH, '//*[@id="j_id_2l:j_id_2p_2_8_1h:dtLawyerParticipantesNovaAudienciaResults_data"]/tr/td').text
                if mensagem != "Nenhum registro encontrado!":
                    #botao excluir
                    self.navegador.find_element(By.ID, 'j_id_2l:j_id_2p_2_8_1h:dtLawyerParticipantesNovaAudienciaResults:0:j_id_2p_2_8_1i_m_1_2_i').click()
            
                time.sleep(1)
                self.navegador.find_element(By.ID, 'j_id_2l:j_id_2p_2_8_1h:autoCompleteLawyer_input').send_keys(advogado)
                
                time.sleep(3)
                self.navegador.find_element(By.XPATH, '//*[@id="j_id_2l:j_id_2p_2_8_1h:autoCompleteLawyer_panel"]/ul/li/span').click()
                
                time.sleep(1)
                self.navegador.find_element(By.XPATH, '//*[@id="j_id_2l:j_id_2p_2_8_1h:dtLawyerParticipantesNovaAudienciaResults:comboAdvogadoResponsavelNovaAudiencia"]/div[3]').click()
                
                time.sleep(1)
                self.navegador.find_element(By.ID, 'j_id_2l:j_id_2p_2_8_1h:dtLawyerParticipantesNovaAudienciaResults:comboAdvogadoResponsavelNovaAudiencia_1').click()
            '''
            
            #Clica em salvar
            time.sleep(2)
            self.navegador.find_element(By.ID, 'btnSalvarNovaAudiencia').click()
            
            #Clicar em Anexo
            time.sleep(2)
            self.navegador.find_element(By.XPATH, '//*[@id="tabViewProcesso"]/ul/li[3]/a').click()

            #time.sleep(2)
            self.navegador.find_element(By.ID, 'tabViewProcesso:j_id_i3_6_1_6_1n').click()

            #Alterna para o popup
            self.navegador.switch_to.frame(self.navegador.find_element(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_6_1_6_1n_dlg"]/div[2]/iframe'))

            #time.sleep(1)
            self.navegador.find_element(By.ID, 'j_id_p:eFileTipoCombo').click()

            time.sleep(2)
            self.navegador.find_element(By.ID, 'j_id_p:eFileTipoCombo_32').click()
        except NoSuchElementException:
            print(Fore.RED + "\tO botão não foi carregado e a página não respondeu... Indo para o próximo.")
            print("------------------------------------------- ")
            self.bandeira = "Erro2"
            self.desc_erro = "O botão de confirmar demorou muito para carregar."
            self.voltaPaginaInicial()
            return
        
        #ElementNotInteractableException
        
        #Baixar arquivos
        time.sleep(1)
        try:
            for pos, obj in enumerate(self.urls):
                print("Baixando o arquivo: ", self.nomes[pos])
                #filename = Path(r"C:\Users\automation\Downloads\{}".format(self.nomes[pos]))
                filename = Path(r"C:\Users\JoséGabrielNevesBuen\Downloads\{}".format(self.nomes[pos]))
                arquivo_pdf = requests.get(self.urls[pos])
                filename.write_bytes(arquivo_pdf.content)
                time.sleep(1)
                #self.navegador.find_element('id', 'j_id_p:j_id_r_2_e_2_1_input').send_keys(r"C:\Users\automation\Downloads\{}".format(self.nomes[pos]))
                self.navegador.find_element(By.ID, 'j_id_p:j_id_r_2_e_2_1_input').send_keys(r"C:\Users\JoséGabrielNevesBuen\Downloads\{}".format(self.nomes[pos]))
        except:
            print(Fore.RED + "O servidor não respondeu ao donwload dos dados... Indo para o próximo")
            self.bandeira = "Erro2"
            self.desc_erro = "O servidor não respondeu ao donwload dos dados."
            self.voltaPaginaInicial()
            return
            
        
        #retorna para a página inicial
        try:
            #Clica em Salvar
            time.sleep(2)
            wait(self.navegador, 10).until(EC.element_to_be_clickable((By.ID, 'j_id_u'))).click()
        
            #Clica em voltar para a página principal
            time.sleep(2)
            wait(self.navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="j_id_2d_1"]/ul/li[2]/a'))).click()
                                            
            time.sleep(2)
            self.navegador.find_element(By.XPATH, '//*[@id="menu-form-contencioso:j_id_2d_a_4"]/a').click()
            
            time.sleep(2)
            self.navegador.find_element(By.ID, 'tabSearchTab:txtSearch').clear()
        except TimeoutException as TME:
            print(Fore.RED + "O servidor demorou muito para responder.")
            self.bandeira = "Erro2"
            self.desc_erro = "O servidor demorou muito para responder"
            self.voltaPaginaInicial()
            return

# Executando o código

In [15]:
#Aciona a classe da API
api = AcessaAPI()
teste = api.post_API()
r = api.get_API(teste) 

tam = len(r)
print("Foram encontrados {} tarefas.".format(tam))

Tentando acessar a API...
	API acessada com sucesso!
Tentando acessar as tarefas da API.
	Token coletado e dados do JSON retornados.
Foram encontrados 59 tarefas.


In [16]:
#Envia email
email = Email(tam)
email.envia_email_sucesso()

In [17]:
#Aciona a classe do Selenium
automacao = Automacao()
automacao.InicializaWebDriver()
automacao.AcessaElaw()
navegador = automacao.RetornaObjeto()

Tentando acessar o navegador...
	Navegador acessado com sucesso!
Acessando o eLaw...
	Elaw acessado com sucesso!


In [18]:
#Percorre o objeto JSON
#Loop principal
bandeira1 = 0
bandeira2 = 0
bandeira3 = 0
bandeira4 = 0
    
for i in range(tam):
    
    if r[i]['flow_item']['item']['data']['tipo_de_providencia'] == "Notificação":
        notificacao = TipoNotificao(i, r, navegador)       
        notificacao.ColetaDados()
        notificacao.MostraDados()
        notificacao.PreencheDados()
  
        if notificacao.return_bandeira() == "Erro1":
            api.tarefa_incompleta(notificacao.return_item_tarefa())
            bandeira2 += 1
            continue
        elif notificacao.return_bandeira() == "Erro2":
            api.tarefa_erro(notificacao.return_item_tarefa(), notificacao.return_desc_erro())
            bandeira3 += 1
            continue
        else:
            bandeira1 += 1
            api.altera_status(notificacao.return_id_tarefa())
            api.tarefa_completa(notificacao.return_id_tarefa(), notificacao.return_id_elaw())

    elif r[i]['flow_item']['item']['data']['tipo_de_providencia'] == "Audiência":
        audiencia = TipoAudiencia(i, r, navegador)
        audiencia.ColetaDados()
        audiencia.MostraDados()
        audiencia.PreencheDados()
        
        if audiencia.return_bandeira() == "Erro1":
            api.tarefa_incompleta(audiencia.return_item_tarefa())
            bandeira2 += 1
            continue
        elif audiencia.return_bandeira() == "Erro2":
            api.tarefa_erro(notificacao.return_item_tarefa(), notificacao.return_desc_erro())
            bandeira3 += 1
            continue
        elif audiencia.return_bandeira() == "Erro3":
            print("Tarefa duplicada...")
            api.tarefa_duplicada(audiencia.return_id_tarefa(), audiencia.return_id_elaw())
            bandeira4 +=1
        else:
            bandeira1 += 1
            api.altera_status(audiencia.return_id_tarefa())
            api.tarefa_completa(audiencia.return_id_tarefa(), audiencia.return_id_elaw())

            
#email.envia_email_fim(bandeira1, bandeira2, bandeira3)
print(Fore.GREEN + "\tForam executados {} tarefas com sucesso! Encerrando o navegador.".format(tam))
print(Fore.GREEN + "O ALGORITMO ESTÁ SENDO EXECUTADO...")
navegador.quit()


Dados de NOTIFICAÇÃO coletados com sucesso!

Referência do Item:  TRISU974D0043-2C18-44A7-B455-B1D
Tipo de providência:  Notificação
ID da Tarefa:  148000
Status da Tarefa:  waiting
Task:  Analisar Documento - 15 dias
Tipo de processo:  Judicial
Número da reclamação procon/numero processo:  1006975-65.2023.8.26.0438
Prazo:  15 dias
Nome do arquivo:  ['Scan2023-07-27_145400.pdf']
URls:  ['https://enlighten.nyc3.digitaloceanspaces.com/api-dev/enspace4c4c/mercado-livre/fields/anexar_notificacao/0UID0000917/e4801b81fe673f8ef3dfd791b62d361540ce21b3e81257b2da67cd5d51c02f35.pdf']
4257534 27/07/2023 15:15 ALERTA Elaborar subsídios ANDERSON RODIGHERI HIRT MATOS Elaborar subsídios
Ver
Editar
Confirmar
Prorrogar
Definir aguardando informações
Cancelar
Trocar Advogados
Anexos
Baixando o arquivo:  Scan2023-07-27_145400.pdf

Alterando o status da tarefa...
Resposta HTTP:  200
	Status da tarefa alterado: Waiting -> Working.

Retornando dados para a API...
Resposta HTTP:  200
	Os dados foram salvos c

URls:  ['https://enlighten.nyc3.digitaloceanspaces.com/api-dev/enspace4c4c/mercado-livre/fields/anexar_notificacao/0UID0000908/7f4ae49c5b3c24b26993a36d4e0fdeb59065559ffaced91a4cf92f0b018e7c2c.pdf', 'https://enlighten.nyc3.digitaloceanspaces.com/api-dev/enspace4c4c/mercado-livre/fields/anexar_notificacao/0UID0000908/99e04f949836a54ad33eb7bbddba6d0a9afaf6a5735462ae52445b7c4e9fa2b1.pdf']

Não foi encontrado nenhum registro para o ID:  147931
------------------------------------------- 
Resposta HTTP:  200

Dados de NOTIFICAÇÃO coletados com sucesso!

Referência do Item:  TRIEX71A56EE2-862C-479F-945C-B29
Tipo de providência:  Notificação
ID da Tarefa:  147921
Status da Tarefa:  waiting
Task:  Analisar Documento - 10 dias
Tipo de processo:  Judicial
Número da reclamação procon/numero processo:  0001970-51.2023.8.05.0022
Prazo:  10 dias
Nome do arquivo:  ['000197051.2023.8.05.00220.pdf', '0001970-51.2023.8.05.0022 (1).pdf']
URls:  ['https://enlighten.nyc3.digitaloceanspaces.com/api-dev/enspa

ID da Tarefa:  147730
Status da Tarefa:  waiting
Task:  Analisar Documento - Acima de 15 dias
Tipo de processo:  Judicial
Número da reclamação procon/numero processo:  0010403-50.2023.8.16.0018
Prazo:  Acima de 15 dias
Nome do arquivo:  ['0010403-50.2023.8.16.0018.pdf']
URls:  ['https://nyc3.digitaloceanspaces.com/enlighten/api-dev/enspace4c4c/mercado-livre/fields/anexar_notificacao/0UID0001026/62b38d6125b6c4f8ef2ea7b5d303bdb1349c1ebd3a8522dee4ee10e1e289b48f.pdf']

Não foi encontrado nenhum registro para o ID:  147730
------------------------------------------- 
Resposta HTTP:  200

Dados de NOTIFICAÇÃO coletados com sucesso!

Referência do Item:  TRIGUBE9FF7EC-6DA5-43BA-878E-C23
Tipo de providência:  Notificação
ID da Tarefa:  147728
Status da Tarefa:  waiting
Task:  Analisar Documento - Acima de 15 dias
Tipo de processo:  Judicial
Número da reclamação procon/numero processo:  0001360-05.2023.8.16.0046
Prazo:  Acima de 15 dias
Nome do arquivo:  ['0001360-05.2023.8.16.0046.pdf']
URls: 

ElementClickInterceptedException: Message: element click intercepted: Element <label id="j_id_2n_label" class="ui-selectonemenu-label ui-inputfield ui-corner-all">...</label> is not clickable at point (583, 283). Other element would receive the click: <div id="acionarWorkflowDialog_modal" class="ui-widget-overlay ui-dialog-mask" style="z-index: 1003;"></div>
  (Session info: chrome=114.0.5735.248)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0063A813+48355]
	(No symbol) [0x005CC4B1]
	(No symbol) [0x004D5358]
	(No symbol) [0x00505062]
	(No symbol) [0x00503B98]
	(No symbol) [0x0050239B]
	(No symbol) [0x0050179E]
	(No symbol) [0x004F9F5C]
	(No symbol) [0x0051A73C]
	(No symbol) [0x004F9A36]
	(No symbol) [0x0051AA94]
	(No symbol) [0x0052C922]
	(No symbol) [0x0051A536]
	(No symbol) [0x004F82DC]
	(No symbol) [0x004F93DD]
	GetHandleVerifier [0x0089AABD+2539405]
	GetHandleVerifier [0x008DA78F+2800735]
	GetHandleVerifier [0x008D456C+2775612]
	GetHandleVerifier [0x006C51E0+616112]
	(No symbol) [0

In [ ]:
'''
#Encontra o escritório    
escritorio_advogado = {
    'GONDIM ADVOGADOS':'Liana Lopes Martins',
    'CHALFIN, GOLDBERG E VAINBOIM':'Cristina Tsiftzoglou',
    'DANNEMANN SIEMSEN ADVOGADOS':'Bruno Wermelinger de Oliveira',
    'MARTORELLI ADVOGADOS':'Kamila Costa de Miranda',
    'ASPIS & PALMEIRO DA FONTOURA ADVOGADOS ASSOCIADOS':'Luiza Cardias',
    'VOSGERAU & CUNHA ADVOGADOS ASSOCIADOS':'Bruno Roberto Vosgerau',
    'LIMA FEIGELSON ADVOGADOS':'Laís Arruda Marini',
    'ANDRADE MAIA ADVOGADOS':'Alessandra Nazareth Mottini',
    'OLIVEIRA RAMOS ADVOGADOS':'Alyne Aparecida Guimarães dos Santos',
    'BBL Advogados':'João Pedro Brígido Pinheiro da Silva',
    'FINCH SOLUÇÕES':'DAIANE VIAN DOS SANTOS',
    'Morais Andrade':'Marcella Porcelli',
    'Chalfin - Trabalhista':'Pamella Maria Fernandes Iglesias Silva Abreu',
    'Chalfin - Diligências': 'Layane Dantas Formiga',
    'ERNESTO BORGES ADVOGADOS':'Thaísa Ferreira',
    'GAIA, SILVA, GAEDE & ASSOCIADOS': 'Maria Aline Buratto Aun',
    'Goulart Penteado': 'Victoria Campanha',
    'MACHADO MEYER SENDACZ OPICE': 'Daniela Leme Arca',
    'MURTA GOYANES ADVOGADOS': 'Gabriel Monnerat Cyrino da Gama e Silva',
    'OPICE BLUM BRUNO ABRUSIO VAINZOF': 'Fernanda Martins Miranda',
    'Ouvidoria PROCON': 'Carolina Aguiar Franco Da Veiga',
    'Pimentel Advogados': 'Daniel Cunha Canto Marques',
    'Trench Rossi Watanabe': 'Marcelo Alves de Siqueira',
    'Rangel e Simões':'Mariana Del Monaco'
}

time.sleep(2)
nome_escritorio = navegador.find_elements('xpath', '//*[@id="processoDadosCabecalhoForm"]/table/tbody/tr/td/label')
for escritorio in escritorio_advogado:
    for nome in nome_escritorio:
        if escritorio == nome.text:
            advogado = escritorio_advogado[escritorio]
            break


print(advogado)

time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="tabViewProcesso:j_id_i3_4_1_3_d:dtAgendamentoResults:0:j_id_i3_4_1_3_1g"]').click()

time.sleep(10)
navegador.switch_to.frame(0)

total_advogados = navegador.find_elements(By.XPATH, '//*[@id="dtLawyerParticipantesProcessoResults_data"]/tr')

time.sleep(5)

#botao excluir
for i in range(len(total_advogados)):
    a = 0
    navegador.find_element(By.ID, f'dtLawyerParticipantesProcessoResults:{a}:j_id_1b').click()
    a += 1
    if a == 3:
        a = 0
    time.sleep(1)

navegador.find_element(By.ID, 'dtLawyerParticipantesProcessoResults:autoCompleteLawyer_input').send_keys(advogado)

time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="dtLawyerParticipantesProcessoResults:autoCompleteLawyer_panel"]/ul').click()

time.sleep(1)
navegador.find_element(By.ID, 'comboAdvogadoResponsavelProcesso_label').click()

time.sleep(1)
navegador.find_element(By.ID, 'comboAdvogadoResponsavelProcesso_1').click()

#Confirma
time.sleep(1)
navegador.find_element(By.ID, 'j_id_t').click()
'''